In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    return image, binary

def detect_keypoints_and_descriptors(image):
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute(image, None)
    return keypoints, descriptors

def match_features(descriptors1, descriptors2):
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches

def find_homography(kp1, kp2, matches):
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 2)
    H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    return H

def detect_boxes(binary_image, box_size):
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filtered_contours = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if cv2.contourArea(contour) > box_size and abs(w - h) < 10:
            filtered_contours.append((x, y, w, h))
    return filtered_contours

def track_and_number_boxes(image_paths, box_size):
    previous_image, previous_binary = preprocess_image(image_paths[0])
    previous_keypoints, previous_descriptors = detect_keypoints_and_descriptors(previous_binary)
    boxes = detect_boxes(previous_binary, box_size)
    box_numbers = {i: box for i, box in enumerate(boxes)}

    for i, image_path in enumerate(image_paths[1:], start=1):
        current_image, current_binary = preprocess_image(image_path)
        current_keypoints, current_descriptors = detect_keypoints_and_descriptors(current_binary)
        matches = match_features(previous_descriptors, current_descriptors)
        H = find_homography(previous_keypoints, current_keypoints, matches)

        for box_number, (x, y, w, h) in box_numbers.items():
            pts = np.float32([[x, y], [x + w, y], [x + w, y + h], [x, y + h]]).reshape(-1, 1, 2)
            dst = cv2.perspectiveTransform(pts, H)
            x, y, w, h = cv2.boundingRect(dst)
            cv2.rectangle(current_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(current_image, str(box_number), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        plt.imshow(cv2.cvtColor(current_image, cv2.COLOR_BGR2RGB))
        plt.title(f"Image {i}")
        plt.show()

        previous_image, previous_binary = current_image, current_binary
        previous_keypoints, previous_descriptors = current_keypoints, current_descriptors

# Example usage
image_paths = ["image1.jpg", "image2.jpg", "image3.jpg"]
box_size = 500  # Adjust based on the size of the boxes
track_and_number_boxes(image_paths, box_size)